# Zimnat Insurance Assurance Challenge

Insurance companies rely on monthly premiums from their clients as their principal source of income; these premiums buy the client insurance against accidents, fires, injury or theft. However, insurance is a competitive market, and there are many factors that can cause a customer to leave an insurance provider, be it poor service delivery, competitive pricing, personal financial stress such, or other environmental factors. This customer loss is known in the business as ‘churn’.

In this hackathon your objective is to develop a predictive model that determines the likelihood for an insurance customer to churn - to seek an alternative insurer or simply drop out of the insurance market altogether.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import utils
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('./raw/train.csv')
clients_df = pd.read_csv('./raw/client_data.csv')
policy_df = pd.read_csv('./raw/policy_data.csv')
payments_df = pd.read_csv('./raw/payment_history.csv')
sample_submission = pd.read_csv('./raw/sample_sub.csv')

In [3]:
train_df.head()

,Policy ID,Lapse,Lapse Year
0,PID_4928TWH,?,?
1,PID_KBLLEGK,?,?
2,PID_90F0QA3,?,?
3,PID_18F3NHF,?,?
4,PID_SX4QUVO,?,?


In [4]:
# Rename some columns in the clients_df and policy_df
train_df.drop(['Lapse Year'], axis=1, inplace=True)

clients_df.rename(columns={'NPH_SEX':'gender', 'NPH_BIRTHDATE':'dob'}, inplace=True)

policy_df.rename(columns={'NP2_EFFECTDATE':'start_date', 'PPR_PRODCD':'product_code', 'NPR_PREMIUM':'amount',
                          'CLF_LIFECD':'principal', 'NLO_TYPE':'premium_type', 'NLO_AMOUNT':'surcharge',
                          'PCL_LOCATCODE':'location', 'OCCUPATION':'occupation', 'CATEGORY':'profession'
                          }, inplace=True
                )

In [6]:
client_mask_columns = ['Policy ID', 'gender', 'dob']
policy_mask_columns = ['Policy ID', 'start_date', 'product_code', 'amount', 'principal',
                        'premium_type', 'surcharge', 'location', 'occupation', 'profession']

clients_df = clients_df[client_mask_columns]
policy_df = policy_df[policy_mask_columns]

# merge client and policy data with train and test dataframes
train_df = train_df.merge(policy_df, on='Policy ID', how='left')
train_df['Lapse'] = train_df['Lapse'].replace(np.nan, 0)
train_df['Lapse'] = train_df['Lapse'].replace('1', 1)
train_df['Lapse'].value_counts()
#train_df = train_df.merge(clients_df, on='Policy ID', how='left')


#train_df.drop_duplicates(inplace=True)
#print(f'Train_shape: {train_df.shape}')
#train_df.head()

?    2622590
1     667175
Name: Lapse, dtype: int64

In [7]:
train_df['year'] = pd.to_datetime(train_df['start_date']).dt.year
train_df['month'] = pd.to_datetime(train_df['start_date']).dt.month
train_df.drop(['start_date'], axis=1, inplace=True)

In [8]:
train_df.head()

,Policy ID,Lapse,product_code,amount,principal,premium_type,surcharge,location,occupation,profession,gender,dob,year,month
0,PID_4928TWH,?,PPR_PRODCD_XRWHUER,42911.077278,1,NLO_TYPE_XAJI0Y6,16702.717882,PCL_LOCATCODE_0T6GYGX,OCCUPATION_9DOM5IG,CATEGORY_8DALFYO,NaN,NaN,2017,1
1,PID_KBLLEGK,?,PPR_PRODCD_64QNIHM,3561.268991,1,NLO_TYPE_XAJI0Y6,600.259636,PCL_LOCATCODE_PI2W0SA,OCCUPATION_8DRC11E,CATEGORY_LXSLG6M,M,1951.0,2018,1
3,PID_KBLLEGK,?,PPR_PRODCD_64QNIHM,3561.268991,1,NLO_TYPE_XAJI0Y6,600.259636,PCL_LOCATCODE_PI2W0SA,OCCUPATION_8DRC11E,CATEGORY_LXSLG6M,M,1986.0,2018,1
7,PID_KBLLEGK,?,PPR_PRODCD_64QNIHM,3561.268991,1,NLO_TYPE_XTHV3A3,776.012205,PCL_LOCATCODE_PI2W0SA,OCCUPATION_8DRC11E,CATEGORY_LXSLG6M,M,1951.0,2018,1
9,PID_KBLLEGK,?,PPR_PRODCD_64QNIHM,3561.268991,1,NLO_TYPE_XTHV3A3,776.012205,PCL_LOCATCODE_PI2W0SA,OCCUPATION_8DRC11E,CATEGORY_LXSLG6M,M,1986.0,2018,1


In [9]:
# Get the test set from the train by splitting
mask = train_df['Lapse'] == '?'
test_df = train_df[mask]
train_df = train_df[~mask]
len(test_df), len(sample_submission), len(train_df)


(224148, 43707, 55712)

In [10]:
test_df.drop_duplicates(subset=['Policy ID'], inplace=True)
train_df['Lapse'].value_counts()

1    55712
Name: Lapse, dtype: int64

In [104]:
payments_df['Policy ID'].nunique(), clients_df['Policy ID'].nunique(), policy_df['Policy ID'].nunique(), test_df['Policy ID'].nunique(), train_df['Policy ID'].nunique()

(32223, 31392, 51685, 43707, 7978)

In [10]:
train_df = train_df.merge(payments_df, on='Policy ID', how='left')
test_df = test_df.merge(payments_df, on='Policy ID', how='left')

train_df.drop_duplicates(inplace=True)
test_df.drop_duplicates(inplace=True)
print(test_df.shape)
test_df.head()

(442809, 18)


,Policy ID,Lapse,product_code,amount,principal,premium_type,surcharge,location,occupation,profession,gender,dob,year,month,AMOUNTPAID,DATEPAID,POSTDATE,PREMIUMDUEDATE
0,PID_4928TWH,?,PPR_PRODCD_XRWHUER,42911.077278,1,NLO_TYPE_XAJI0Y6,16702.717882,PCL_LOCATCODE_0T6GYGX,OCCUPATION_9DOM5IG,CATEGORY_8DALFYO,NaN,NaN,2017,1,50960.151169,2017-07-31 00:00:00,2017-01-08 00:00:00,2017-01-08 00:00:00
1,PID_4928TWH,?,PPR_PRODCD_XRWHUER,42911.077278,1,NLO_TYPE_XAJI0Y6,16702.717882,PCL_LOCATCODE_0T6GYGX,OCCUPATION_9DOM5IG,CATEGORY_8DALFYO,NaN,NaN,2017,1,0.000000,2017-02-08 00:00:00,2017-02-08 00:00:00,2017-01-08 00:00:00
2,PID_4928TWH,?,PPR_PRODCD_XRWHUER,42911.077278,1,NLO_TYPE_XAJI0Y6,16702.717882,PCL_LOCATCODE_0T6GYGX,OCCUPATION_9DOM5IG,CATEGORY_8DALFYO,NaN,NaN,2017,1,50960.464772,2018-06-12 00:00:00,2018-06-12 00:00:00,2018-01-08 00:00:00
3,PID_4928TWH,?,PPR_PRODCD_XRWHUER,42911.077278,1,NLO_TYPE_XAJI0Y6,16702.717882,PCL_LOCATCODE_0T6GYGX,OCCUPATION_9DOM5IG,CATEGORY_8DALFYO,NaN,NaN,2017,1,0.000000,2018-06-12 00:00:00,2018-06-12 22:00:26,2018-01-08 00:00:00
4,PID_KBLLEGK,?,PPR_PRODCD_64QNIHM,3561.268991,1,NLO_TYPE_XAJI0Y6,600.259636,PCL_LOCATCODE_PI2W0SA,OCCUPATION_8DRC11E,CATEGORY_LXSLG6M,M,1951.0,2018,1,4072.569943,2018-04-30 00:00:00,2018-02-05 00:00:00,2018-01-05 00:00:00


In [11]:
test_df.drop_duplicates(subset=['Policy ID'], inplace=True)
#train_df.drop_duplicates(subset=['Policy ID'], inplace=True)
train_df.shape, test_df.shape

((326092, 18), (43707, 18))

In [12]:
# create processed data dir and save new train and test dataframes
utils.makedir('./proc_data')
train_df.to_csv('./proc_data/train.csv', index=False)
test_df.to_csv('./proc_data/test.csv', index=False)

Path ./proc_data already exists!


## Building the Model and Evaluation

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import log_loss
import xgboost as xgb


In [14]:
train_data = pd.read_csv('./proc_data/train.csv')
test_data = pd.read_csv('./proc_data/test.csv')
sample_submission = pd.read_csv('./raw/sample_sub.csv')

In [15]:
#train_data.drop_duplicates(subset=['Policy ID'], inplace=True)
train_data.head()

,Policy ID,Lapse,product_code,amount,principal,premium_type,surcharge,location,occupation,profession,gender,dob,year,month,AMOUNTPAID,DATEPAID,POSTDATE,PREMIUMDUEDATE
0,PID_MFAAYNJ,1,PPR_PRODCD_KOFUYNN,1074.485839,4,NLO_TYPE_DPBHSAH,NaN,PCL_LOCATCODE_SPQHMX5,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,F,1987.0,2018,1,0.000000,2018-01-10 00:00:00,2018-01-10 00:00:00,2018-01-10 00:00:00
1,PID_MFAAYNJ,1,PPR_PRODCD_KOFUYNN,1074.485839,4,NLO_TYPE_DPBHSAH,NaN,PCL_LOCATCODE_SPQHMX5,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,F,1987.0,2018,1,3517.916006,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-01-10 00:00:00
2,PID_MFAAYNJ,1,PPR_PRODCD_KOFUYNN,1288.734649,1,NLO_TYPE_XAJI0Y6,1339.461987,PCL_LOCATCODE_SPQHMX5,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,F,1987.0,2018,1,0.000000,2018-01-10 00:00:00,2018-01-10 00:00:00,2018-01-10 00:00:00
3,PID_MFAAYNJ,1,PPR_PRODCD_KOFUYNN,1288.734649,1,NLO_TYPE_XAJI0Y6,1339.461987,PCL_LOCATCODE_SPQHMX5,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,F,1987.0,2018,1,3517.916006,2018-09-29 00:00:00,2018-09-29 00:00:00,2018-01-10 00:00:00
4,PID_MFAAYNJ,1,PPR_PRODCD_KOFUYNN,1008.996236,2,NLO_TYPE_DPBHSAH,NaN,PCL_LOCATCODE_SPQHMX5,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,F,1987.0,2018,1,0.000000,2018-01-10 00:00:00,2018-01-10 00:00:00,2018-01-10 00:00:00


In [16]:
train_data['age'] = (2020 - train_data['dob'])
train_data['age'] = train_data['age'].fillna(train_data['age'].mean())
test_data['age'] = (2020 - test_data['dob'])
test_data['age'] = test_data['age'].fillna(test_data['age'].mean())

train_data['surcharge'] = train_data['surcharge'].fillna(0)
test_data['surcharge'] = test_data['surcharge'].fillna(0)

In [19]:
train_data['premium_type'].nunique(), test_data['premium_type'].nunique()

(5, 6)

In [18]:
categorical_features = ['profession']
for col in categorical_features:
    encoder = LabelEncoder()
    train_data[col] = encoder.fit_transform(train_data[col])
    test_data[col] = encoder.transform(test_data[col])

In [27]:
test_data.head()

,Policy ID,Lapse,product_code,amount,principal,premium_type,surcharge,location,occupation,profession,gender,dob,year,month,AMOUNTPAID,DATEPAID,POSTDATE,PREMIUMDUEDATE,age
0,PID_4928TWH,?,PPR_PRODCD_XRWHUER,42911.077278,1,NLO_TYPE_XAJI0Y6,16702.717882,PCL_LOCATCODE_0T6GYGX,OCCUPATION_9DOM5IG,1,NaN,NaN,2017,1,50960.151169,2017-07-31 00:00:00,2017-01-08 00:00:00,2017-01-08 00:00:00,38.247066
1,PID_KBLLEGK,?,PPR_PRODCD_64QNIHM,3561.268991,1,NLO_TYPE_XAJI0Y6,600.259636,PCL_LOCATCODE_PI2W0SA,OCCUPATION_8DRC11E,3,M,1951.0,2018,1,4072.569943,2018-04-30 00:00:00,2018-02-05 00:00:00,2018-01-05 00:00:00,69.000000
2,PID_90F0QA3,?,PPR_PRODCD_64QNIHM,6164.812836,1,NLO_TYPE_XTHV3A3,1343.332956,PCL_LOCATCODE_O6OBSFL,OCCUPATION_NNHJ7XV,2,F,2010.0,2019,1,NaN,NaN,NaN,NaN,10.000000
3,PID_18F3NHF,?,PPR_PRODCD_GLE2MHV,2278.189789,1,NLO_TYPE_XAJI0Y6,1318.742708,PCL_LOCATCODE_295LOO6,OCCUPATION_NNHJ7XV,2,F,1960.0,2019,1,NaN,NaN,NaN,NaN,60.000000
4,PID_SX4QUVO,?,PPR_PRODCD_W0F6GK1,1619.046308,1,NLO_TYPE_XAJI0Y6,332.816358,PCL_LOCATCODE_PEU5TF2,OCCUPATION_NNHJ7XV,2,M,2006.0,2019,1,NaN,NaN,NaN,NaN,14.000000


In [28]:
target = train_data['Lapse']
use_columns = ['product_code', 'amount', 'principal', 'premium_type', 'surcharge',
               'location', 'occupation', 'profession', 'age']

features = ['amount', 'principal', 'surcharge', 'profession', 'age', 'month', 'year']
train_features = train_data[features]
train_features.fillna(train_features.mean(), inplace=True)
test_features = test_data[features]
test_features.fillna(test_features.mean(), inplace=True)

In [80]:
folds = KFold(3, shuffle=False)
scores = []
for train_index, val_index in folds.split(train_features, target):
    X_train, y_train = train_features.loc[train_index], target.loc[train_index]
    X_val, y_val = train_features.loc[val_index], target.loc[val_index]

    model = xgb.XGBClassifier(max_depth=3,
                             n_estimators=170,
                             #colsample_bytree=0.8,
                             objective='reg:squarederror',
                             #subsample=0.8,
                             metric='logloss',
                             nthread=2,
                             learning_rate=0.01,
                             random_state=77#42
                             )
    
    model.fit(X_train, y_train)
    score = log_loss(y_val, model.predict_proba(X_val)[:,1], labels=[0, 1])
    scores.append(score)
    print(score)
print(f'Mean_RMSE: {np.mean(scores)}')

2.4016966819763184
2.4016966819763184
2.4016966819763184
Mean_RMSE: 2.4016966819763184


In [81]:
predictions = model.predict_proba(test_features)[:,1]

In [82]:
min(predictions), max(predictions)

(0.09056416, 0.09056416)

In [83]:
sample_submission['Lapse'] = predictions

In [84]:
utils.makedir('./submissions')
sample_submission.to_csv(f'./submissions/sub_xgb-clf_{np.round(np.mean(scores), 4)}', index=False)
#sample_submission.to_csv(f'./submissions/sub_svr_{np.round(np.mean(scores), 4)}', index=False)

Path ./submissions already exists!


In [85]:
sample_submission.head()

,Policy ID,Lapse
0,PID_4928TWH,0.090564
1,PID_KBLLEGK,0.090564
2,PID_90F0QA3,0.090564
3,PID_18F3NHF,0.090564
4,PID_SX4QUVO,0.090564
